<a href="https://colab.research.google.com/github/Patisson4/ML/blob/lab-5/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

In [8]:
import pandas as pd
import nltk

data = pd.read_csv(
    'https://github.com/sismetanin/rureviews/blob/master/women-clothing-accessories.3-class.balanced.csv?raw=true',
    sep='\t')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mdpol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mdpol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
import string
import numpy as np
from time import perf_counter
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier  # можно заменить на другой классификатор
from sklearn.naive_bayes import MultinomialNB  # наивный байесовский классификатор
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

In [ ]:
data['format'] = data['review']

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [14]:
start = perf_counter()

analyzer = MorphAnalyzer()

for i in range(len(data)):
    for pun in string.punctuation:
        data.format[i] = data.format[i].replace(pun, "")
    tokens = [analyzer.parse(word.lower())[0].normal_form for word in word_tokenize(data.format[i])]
    data.format[i] = " ".join(tokens)

print(data.head(10))
print(f'Performed in {perf_counter() - start}')

                                              review sentiment  \
0  качество плохое пошив ужасный (горловина напер...  negative   
1  Товар отдали другому человеку, я не получила п...  negative   
2  Ужасная синтетика! Тонкая, ничего общего с пре...  negative   
3  товар не пришел, продавец продлил защиту без м...  negative   
4      Кофточка голая синтетика, носить не возможно.  negative   
5                              Очень глубокие проймы  negative   
6  Я недовольна заказом.Я вот одного не понимаю п...  negative   
7  заказала размер s на от 64,об 94,начнем с того...  negative   
8  Заказ я сделала в июле. С тех пор посылка отсл...  negative   
9                           Ужасное качество товара!  negative   

                                              format  
0  [качество, плохой, пошив, ужасный, горловина, ...  
1  [товар, отдать, другой, человек, я, не, получи...  
2  [ужасный, синтетик, тонкий, ничего, общий, с, ...  
3  [товар, не, прийти, продавец, продлить, защита... 

In [32]:
results = []
X_train, X_test, y_train, y_test = train_test_split(data.format, data.sentiment, train_size=0.7)

In [29]:
MIN_NGRAM = 2
MAX_NGRAM = 8

for min_n in range(MIN_NGRAM, MAX_NGRAM):
    for max_n in range(min_n, MAX_NGRAM):
        start = perf_counter()

        count_vectorizer = CountVectorizer(ngram_range=(min_n, max_n), stop_words=stopwords.words('russian'), analyzer='word')

        classifier = MultinomialNB()
        classifier.fit(count_vectorizer.fit_transform(X_train), y_train)

        predicted = classifier.predict(count_vectorizer.transform(X_test))

        results.append([classification_report(y_test, predicted, output_dict=True), count_vectorizer])

        print("ngram_range:", str(count_vectorizer.ngram_range), "analyzer:", str(count_vectorizer.analyzer), classification_report(y_test, predicted, output_dict=False))
        print(f'Performed in {perf_counter() - start}')

ngram_range: (2, 2) analyzer: word
              precision    recall  f1-score   support

    neautral       0.59      0.51      0.55      9030
    negative       0.70      0.65      0.67      9080
    positive       0.71      0.86      0.78      8891

    accuracy                           0.67     27001
   macro avg       0.67      0.67      0.67     27001
weighted avg       0.67      0.67      0.67     27001

Performed in 6.21522479999112
ngram_range: (2, 3) analyzer: word
              precision    recall  f1-score   support

    neautral       0.59      0.49      0.54      9030
    negative       0.69      0.65      0.67      9080
    positive       0.70      0.86      0.77      8891

    accuracy                           0.67     27001
   macro avg       0.66      0.67      0.66     27001
weighted avg       0.66      0.67      0.66     27001

Performed in 9.461641200003214
ngram_range: (2, 4) analyzer: word
              precision    recall  f1-score   support

    neautral     

In [33]:
MIN_NGRAM = 2
MAX_NGRAM = 4

for min_n in range(MIN_NGRAM, MAX_NGRAM):
    for max_n in range(min_n, MAX_NGRAM):
        for max_df in [0.1, 0.5]:
            for min_df in [0, 0.01]:
                for max_features in [2048, 8192, 32768]:
                    start = perf_counter()

                    tfidf_vectorizer = TfidfVectorizer(ngram_range=(min_n, max_n), stop_words=stopwords.words('russian'), max_df=max_df, min_df=min_df, max_features=max_features)

                    classifier = MultinomialNB()
                    classifier.fit(tfidf_vectorizer.fit_transform(X_train), y_train)

                    predicted = classifier.predict(tfidf_vectorizer.transform(X_test))

                    results.append([classification_report(y_test, predicted, output_dict=True), tfidf_vectorizer])

                    print("ngram_range:", str(tfidf_vectorizer.ngram_range), "analyzer:", str(tfidf_vectorizer.analyzer), classification_report(y_test, predicted, output_dict=False))
                    print(f'Performed in {perf_counter() - start}')

ngram_range: (2, 2) analyzer: word {'neautral': {'precision': 0.5537627297976458, 'recall': 0.4636766334440753, 'f1-score': 0.5047314809233922, 'support': 9030}, 'negative': {'precision': 0.7195517546446476, 'recall': 0.5374449339207048, 'f1-score': 0.6153070230740134, 'support': 9080}, 'positive': {'precision': 0.5930636751461527, 'recall': 0.8443369699696323, 'f1-score': 0.6967376676411899, 'support': 8891}, 'accuracy': 0.6138291174400948, 'macro avg': {'precision': 0.6221260531961487, 'recall': 0.6151528457781374, 'f1-score': 0.6055920572128652, 'support': 27001}, 'weighted avg': {'precision': 0.6224560763664525, 'recall': 0.6138291174400948, 'f1-score': 0.605140833496837, 'support': 27001}}
Performed in 4.777939999999944
ngram_range: (2, 2) analyzer: word {'neautral': {'precision': 0.5706508589402371, 'recall': 0.5223698781838316, 'f1-score': 0.5454440333024977, 'support': 9030}, 'negative': {'precision': 0.7175479135517194, 'recall': 0.5813876651982379, 'f1-score': 0.6423313256677

C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

ngram_range: (3, 3) analyzer: word {'neautral': {'precision': 0.20833333333333334, 'recall': 0.004429678848283499, 'f1-score': 0.008674907829104316, 'support': 9030}, 'negative': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9080}, 'positive': {'precision': 0.33126934984520123, 'recall': 0.998875267124058, 'f1-score': 0.49753501400560224, 'support': 8891}, 'accuracy': 0.3303951705492389, 'macro avg': {'precision': 0.17986756105951152, 'recall': 0.33443498199078053, 'f1-score': 0.16873664061156887, 'support': 27001}, 'weighted avg': {'precision': 0.17875507534808652, 'recall': 0.3303951705492389, 'f1-score': 0.16673153687717573, 'support': 27001}}
Performed in 6.760216800001217


C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

ngram_range: (3, 3) analyzer: word {'neautral': {'precision': 0.20833333333333334, 'recall': 0.004429678848283499, 'f1-score': 0.008674907829104316, 'support': 9030}, 'negative': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9080}, 'positive': {'precision': 0.33126934984520123, 'recall': 0.998875267124058, 'f1-score': 0.49753501400560224, 'support': 8891}, 'accuracy': 0.3303951705492389, 'macro avg': {'precision': 0.17986756105951152, 'recall': 0.33443498199078053, 'f1-score': 0.16873664061156887, 'support': 27001}, 'weighted avg': {'precision': 0.17875507534808652, 'recall': 0.3303951705492389, 'f1-score': 0.16673153687717573, 'support': 27001}}
Performed in 7.994243399996776


...

C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

ngram_range: (3, 3) analyzer: word {'neautral': {'precision': 0.20833333333333334, 'recall': 0.004429678848283499, 'f1-score': 0.008674907829104316, 'support': 9030}, 'negative': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9080}, 'positive': {'precision': 0.33126934984520123, 'recall': 0.998875267124058, 'f1-score': 0.49753501400560224, 'support': 8891}, 'accuracy': 0.3303951705492389, 'macro avg': {'precision': 0.17986756105951152, 'recall': 0.33443498199078053, 'f1-score': 0.16873664061156887, 'support': 27001}, 'weighted avg': {'precision': 0.17875507534808652, 'recall': 0.3303951705492389, 'f1-score': 0.16673153687717573, 'support': 27001}}
Performed in 6.9064738000015495


C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

ngram_range: (3, 3) analyzer: word {'neautral': {'precision': 0.20833333333333334, 'recall': 0.004429678848283499, 'f1-score': 0.008674907829104316, 'support': 9030}, 'negative': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9080}, 'positive': {'precision': 0.33126934984520123, 'recall': 0.998875267124058, 'f1-score': 0.49753501400560224, 'support': 8891}, 'accuracy': 0.3303951705492389, 'macro avg': {'precision': 0.17986756105951152, 'recall': 0.33443498199078053, 'f1-score': 0.16873664061156887, 'support': 27001}, 'weighted avg': {'precision': 0.17875507534808652, 'recall': 0.3303951705492389, 'f1-score': 0.16673153687717573, 'support': 27001}}
Performed in 7.004192100008368


C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [2]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb')
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [20]:
result = re.findall(r'\b\w{2}', "Mauris id augue ac risus convallis laoreet a eu tellus")
print(result)

['Ma', 'id', 'au', 'ac', 'ri', 'co', 'la', 'eu', 'te']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [7]:
result = re.split('\.',
                  'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque tincidunt ac nulla sed pellentesque. Fusce non pellentesque elit, ac viverra est. Maecenas laoreet, leo eget venenatis dictum, nisi eros vestibulum sem, nec tristique est risus et lorem. Cras faucibus lorem ante, sed volutpat est hendrerit fermentum. Vestibulum aliquam non lectus eu blandit. Nam suscipit lacus id sollicitudin aliquam. Etiam ut ultricies mauris.',
                  maxsplit=2)
print(result)

['Lorem ipsum dolor sit amet, consectetur adipiscing elit', ' Pellentesque tincidunt ac nulla sed pellentesque', ' Fusce non pellentesque elit, ac viverra est. Maecenas laoreet, leo eget venenatis dictum, nisi eros vestibulum sem, nec tristique est risus et lorem. Cras faucibus lorem ante, sed volutpat est hendrerit fermentum. Vestibulum aliquam non lectus eu blandit. Nam suscipit lacus id sollicitudin aliquam. Etiam ut ultricies mauris.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print(result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [55]:
result = re.sub(r'\d', 'DIG', '1 one 2 two 3 three 4 four 10 ten')
print(result)

DIG one DIG two DIG three DIG four DIGDIG ten


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [37]:
result = re.sub(r'\w+://\S+', '[link removed]',
                'Docs: https://docs.python.org/3/library/re.html Meme: https://www.youtube.com/watch?v=dQw4w9WgXcQ&t=0s')
print(result)

Docs: [link removed] Meme: [link removed]


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [46]:
rg = re.compile('\w{4,}')
rg.findall("Vestibulum pellentesque facilisis orci, id convallis dui vulputate id.")

['Vestibulum', 'pellentesque', 'facilisis', 'orci', 'convallis', 'vulputate']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [54]:
re.findall(r'@[\w.]+', 'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']